In [1]:
import pandas as pd
import numpy as np
import math

In [7]:
df = pd.read_csv(r"coverage.csv",sep=',')
df

,Unnamed: 0,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,r
0,t1,1,0,0,0,1,1,0,1,1,1,1
1,t2,1,1,0,1,0,0,0,0,1,0,0
2,t3,1,0,1,0,0,1,1,1,1,1,1
3,t4,1,1,1,1,0,0,1,1,1,0,0
4,t5,1,0,0,0,0,0,0,0,0,0,0
5,t6,1,0,0,0,0,0,0,0,0,0,0
6,t7,1,0,0,0,0,0,0,0,0,0,0
7,t8,1,0,0,0,0,0,0,0,0,0,0
8,t9,0,0,0,0,0,0,0,0,0,0,0
9,t10,1,0,0,0,0,0,0,0,0,0,1


In [8]:
class Tarantula(object):

    def __init__(self, directory):

        self.__M = []  # boolean. coverage matrix
        self.__MT = []
        self.__F = []  # boolean. failing test cases
        self.__C = []
        self.__numOrigTests = 0
        self.__numStmts = 0
        self.__totalOrigFail = 0
        self.__totalOrigPass = 0
        self.__passOnStmt = []  # int. p(s), for every s and considering liveness of test cases
        self.__failOnStmt = []  # int. p(s), for every s and considering liveness of test cases
        self.__passRatio = []
        self.__failRatio = []
        self.__suspiciousness = []
        self.__directory = directory  # to store the coverage matrix file
        self.__ranking = []



    def readFile(self):
        self.__M = pd.read_csv(self.__directory, sep=',', index_col=0)

        # initialize all case are live
        self.TarantulaSuspiciousnessCalculation(self.__M)

        # initialize the fail and pass case number
        self.setF(self.__M.r)
        # print(self.__F)

    def TarantulaSuspiciousnessCalculation(self, M):
        self.__M = M

        self.__numOrigTests = self.__M.shape[0]
        # print(self.__numOrigTests)
        self.__numStmts = self.__M.shape[1] - 1
        # print(self.__numStmts)


    def setDirectort(self, dir):
        self.__directory = dir

    def setC(self, C):
        self.__C = C

    def setF(self, F):
        self.__F = F.tolist()
        self.calculateOrigFailAndPass()

    def getM(self):
        return self.__M

    def getF(self):
        return self.__F

    def getPassOnStmt(self):
        return self.__passOnStmt

    def getFailOnStmt(self):
        return self.__failOnStmt


    def mainCompute(self):
        self.calculatePassOnStmtAndFailOnStmt()
        self.calculatePassRatioAndFailRatio()
        self.calculateSuspiciousness()
        self.calculateRanking()

    def getSuspiciousness(self):
        return self.__suspiciousness

    def getOrigNumTests(self):
        return self.__numOrigTests

    def getOrigTotalFail(self):
        return self.__totalOrigFail

    def getOrigTotalPass(self):
        return self.__totalOrigPass

    def getNumStmts(self):
        return self.__numStmts

    def calculatePassOnStmtAndFailOnStmt(self):
        for i in range(self.__numStmts):
            self.__C = self.__M.iloc[:, i]
            self.__failOnStmt.insert(i, self.__C.tolist().count(0))
            self.__passOnStmt.insert(i, self.__C.tolist().count(1))

    def calculatePassRatioAndFailRatio(self):



        for i in range(self.__numStmts):
            if self.__totalOrigPass == 0:
                self.__passRatio.insert(i, 0)
            else:
                self.__passRatio.insert(i, self.__passOnStmt[i] / self.__totalOrigPass)

            if self.__totalOrigPass == 0:
                self.__failRatio.insert(i, 0)
            else:

                self.__failRatio.insert(i, self.__failOnStmt[i] / self.__totalOrigFail)

    def calculateSuspiciousness(self):
        for i in range(self.__numStmts):
            if (self.__totalOrigFail == 0) & (self.__totalOrigPass == 0):
                self.__suspiciousness.insert(i, -1)
            elif (self.__failRatio[i] == 0) & (self.__passRatio[i] == 0):
                self.__suspiciousness.insert(i, -1)
            else:
                self.__suspiciousness.insert(i, round(self.__failRatio[i] / (self.__failRatio[i] + self.__passRatio[i]), 2))

    def calculateOrigFailAndPass(self):
        for i in range(self.__numOrigTests):
            if self.__F[i]:

                self.__totalOrigPass += 1
            else:

                self.__totalOrigFail += 1

    def calculateRanking(self):
        # print whole list with suspiciousness and rank
        self.__MT = self.__M.T
        self.__suspiciousness.append(np.nan)
        self.__suspiciousness = pd.Series(self.__suspiciousness, index=[var for var in self.__M.columns])
        self.__ranking = self.__suspiciousness.rank(ascending=False, method="min")

        self.__MT["Suspiciousness"] = self.__suspiciousness
        self.__MT['Rank'] = self.__ranking
        self.__MT.sort_values("Rank", inplace=True)

        print(self.__MT)

In [10]:
if __name__ == "__main__":
    file = r"coverage.csv"
    cal = Tarantula(file)
    cal.readFile()
    cal.mainCompute()

     t1  t2  t3  t4  t5  t6  t7  t8  t9  t10  ...   t29  t30  t31  t32  t33  \
s5    1   0   0   0   0   0   0   0   0    0  ...     0    0    0    0    0   
s2    0   1   0   1   0   0   0   0   0    0  ...     0    0    0    0    0   
s3    0   0   1   1   0   0   0   0   0    0  ...     0    0    0    0    0   
s4    0   1   0   1   0   0   0   0   0    0  ...     0    0    0    0    0   
s6    1   0   1   0   0   0   0   0   0    0  ...     0    0    0    0    0   
s7    0   0   1   1   0   0   0   0   0    0  ...     0    0    0    0    0   
s10   1   0   1   0   0   0   0   0   0    0  ...     0    0    0    0    0   
s8    1   0   1   1   0   0   0   0   0    0  ...     0    0    0    0    0   
s9    1   1   1   1   0   0   0   0   0    0  ...     0    0    0    0    0   
s1    1   1   1   1   1   1   1   1   0    1  ...     0    1    1    1    1   
r     1   0   1   0   0   0   0   0   0    1  ...     0    0    1    0    0   

     t34  t35  t36  Suspiciousness  Rank  
s5     0